# Proyecto Wine — Notebook generado a partir de tu script modificado
Este cuaderno contiene el código actualizado con los cambios solicitados:
- `penalty='none'` en lugar de `None` para `LogisticRegression`.
- Importación de `display` desde IPython si ejecutas fuera de notebook.
- Inserción de `error_score=np.nan` en `RandomizedSearchCV` para mayor robustez.
- Eliminación de variable `y_test_bin` sin uso.


In [ ]:
from IPython.display import display
# -*- coding: utf-8 -*-
"""Proyecto_Wine_Act2_Mod11_v2.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1zKFZfRF7AXHaHZhws6beOe8uVfqfZ1fL

# Clasificación de Vinos (Wine Dataset)

**Curso:** Machine Learning — Actividad 2 - Modulo 11  
**Autor:** _John Gómez_  
**Fecha:** _2025-10-01_

---

## Propósito del cuaderno
Este notebook implementa un flujo completo de clasificación supervisada sobre el dataset **Wine** (scikit-learn), que incluye:
1) EDA avanzada con correlaciones y análisis de outliers.  
2) Búsqueda automatizada de hiperparámetros (GridSearchCV/RandomizedSearchCV).  
3) Análisis de importancia de características.  
4) Documentación completa de decisiones técnicas.  
5) Pipeline reproducible y listo para portafolio.
"""

# Commented out IPython magic to ensure Python compatibility.
# ==== Configuración para Google Colab (opcional) ====
# Ejecuta esta celda SOLO si estás en Colab o te faltan librerías.
# En tu entorno local, puedes comentar esta celda.
try:
    # Intenta importar el módulo google.colab para detectar si se está ejecutando en Colab.
    import google.colab  # type: ignore
    # Si estamos en Colab, instala las librerías necesarias usando pip.
    # -q suprime la salida detallada de pip.
#     %pip install -q scikit-learn matplotlib pandas joblib seaborn
except Exception:
    # Si ocurre una excepción (no estamos en Colab o ya están instaladas), no hacemos nada.
    pass
# Imprime un mensaje para confirmar que el entorno está listo.
print("Entorno listo ✅")

# ==== 1. Importación de librerías ====
import os # Para interactuar con el sistema operativo (crear directorios, etc.)
import json # Para trabajar con datos en formato JSON (guardar metadatos)
import joblib # Para guardar y cargar modelos de scikit-learn de forma eficiente
import numpy as np # Para operaciones numéricas y arrays multidimensionales
import pandas as pd # Para manipulación y análisis de datos (DataFrames)
import matplotlib.pyplot as plt # Para crear visualizaciones estáticas
import seaborn as sns # Basado en matplotlib, para visualizaciones estadísticas más atractivas
from scipy import stats # Para funciones estadísticas (ej: skewness)

# Scikit-learn imports
from sklearn.datasets import load_wine # Carga el dataset de vinos
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV # Para dividir datos, validación cruzada y búsqueda de hiperparámetros
from sklearn.preprocessing import StandardScaler, LabelEncoder # Para escalar características y codificar etiquetas
from sklearn.pipeline import Pipeline # Para encadenar pasos de procesamiento de datos y modelado
from sklearn.linear_model import LogisticRegression # Modelo de Regresión Logística
from sklearn.ensemble import RandomForestClassifier # Modelo de Bosque Aleatorio
from sklearn.svm import SVC # Modelo de Máquinas de Vectores de Soporte
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, # Métricas de evaluación: precisión, reporte completo, matriz de confusión
    roc_auc_score, RocCurveDisplay, precision_recall_curve, PrecisionRecallDisplay # Métricas basadas en curvas ROC y Precision-Recall
)
from sklearn.inspection import permutation_importance # Para calcular la importancia de características por permutación

# Configuración de visualización
plt.style.use('seaborn-v0_8') # Aplica un estilo visual a los gráficos
pd.set_option("display.max_columns", 100) # Configura pandas para mostrar hasta 100 columnas
pd.set_option("display.width", 1000) # Configura el ancho de visualización de pandas
sns.set_palette("husl") # Establece la paleta de colores para seaborn

# Confirma que todas las librerías han sido importadas.
print("✅ Todas las librerías importadas correctamente")

"""## 2. Carga de datos y EDA Avanzado
Análisis exploratorio completo con visualizaciones mejoradas.
"""

# Carga del dataset
wine = load_wine()
X = pd.DataFrame(wine.data, columns=wine.feature_names)
y = pd.Series(wine.target, name="class")

# Diccionario de traducción mejorado
column_titles_es = {
    "alcohol": "Contenido de Alcohol",
    "malic_acid": "Ácido Málico",
    "ash": "Ceniza",
    "alcalinity_of_ash": "Alcalinidad de Ceniza",
    "magnesium": "Magnesio",
    "total_phenols": "Fenoles Totales",
    "flavanoids": "Flavonoides",
    "nonflavanoid_phenols": "Fenoles No Flavonoides",
    "proanthocyanins": "Proantocianinas",
    "color_intensity": "Intensidad del Color",
    "hue": "Matiz",
    "od280/od315_of_diluted_wines": "OD280/OD315 de Vinos Diluidos",
    "proline": "Prolina",
}

print("=== INFORMACIÓN GENERAL DEL DATASET ===")
print(f"📊 Dimensiones: {X.shape}")
print(f"🎯 Número de clases: {len(np.unique(y))}")
print(f"🔢 Clases: {wine.target_names.tolist()}")

# Distribución de clases
print("\n=== DISTRIBUCIÓN DE CLASES ===")
class_dist = y.value_counts().sort_index()
for cls, count in class_dist.items():
    print(f"Clase {cls} ({wine.target_names[cls]}): {count} muestras ({count/len(y)*100:.1f}%)")

# Análisis de valores nulos
print("\n=== VALORES NULOS ===")
print(X.isnull().sum())

# Estadísticas descriptivas mejoradas
print("\n=== ESTADÍSTICAS DESCRIPTIVAS ===")
stats_df = X.describe().T
stats_df['cv'] = (stats_df['std'] / stats_df['mean']) * 100  # Coeficiente de variación
stats_df['skew'] = X.apply(stats.skew)
display(stats_df.round(3))

"""### 2.1 Visualizaciones Avanzadas de EDA"""

# 1. Histogramas con distribución por clase
print("\n📈 HISTOGRAMAS POR CARACTERÍSTICA Y CLASE")
fig, axes = plt.subplots(4, 4, figsize=(20, 15))
axes = axes.ravel()

for i, col in enumerate(X.columns[:13]):
    for cls in range(3):
        axes[i].hist(X[y == cls][col], alpha=0.7, label=f'Clase {cls}', bins=15)
    axes[i].set_title(f"{column_titles_es.get(col, col)}\n({col})", fontsize=10)
    axes[i].set_xlabel(col)
    axes[i].legend()

# Eliminar ejes vacíos
for i in range(13, 16):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

# 2. Matriz de correlación
print("\n🔥 MATRIZ DE CORRELACIÓN")
plt.figure(figsize=(12, 10))
correlation_matrix = X.corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, fmt='.2f', cbar_kws={"shrink": .8})
plt.title('Matriz de Correlación entre Características', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

# 3. Boxplots para detectar outliers
print("\n📦 BOXPLOTS POR CARACTERÍSTICA")
fig, axes = plt.subplots(4, 4, figsize=(20, 15))
axes = axes.ravel()

for i, col in enumerate(X.columns[:13]):
    X[col].plot(kind='box', ax=axes[i], vert=False)
    axes[i].set_title(f"{column_titles_es.get(col, col)}", fontsize=10)
    axes[i].set_xlabel('Valor')

for i in range(13, 16):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

# 4. Pairplot de características más importantes
print("\n🔍 PAIRPLOT DE CARACTERÍSTICAS PRINCIPALES")
features_for_pairplot = ['alcohol', 'malic_acid', 'ash', 'flavanoids', 'color_intensity', 'proline']
sample_df = X[features_for_pairplot].copy()
sample_df['class'] = y
sample_df['class_name'] = sample_df['class'].map(lambda x: wine.target_names[x])

sns.pairplot(sample_df, hue='class_name', diag_kind='hist')
plt.suptitle('Pairplot de Características Seleccionadas por Clase', y=1.02)
plt.show()

"""## 3. Partición Train/Test (Estratificada)"""

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

print("=== PARTICIONES DE DATOS ===")
print(f"✅ Conjunto de entrenamiento: {X_train.shape[0]} muestras ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"✅ Conjunto de prueba: {X_test.shape[0]} muestras ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"✅ Proporción de clases en train: {np.bincount(y_train) / len(y_train)}")
print(f"✅ Proporción de clases en test: {np.bincount(y_test) / len(y_test)}")

"""## 4. Definición de Pipelines y Búsqueda de Hiperparámetros"""

# DECISIONES DE DISEÑO:
# 1. Estandarización en LogReg y SVM (sensibles a escala)
# 2. RF sin escalado, mantenemos pipeline por consistencia
# 3. Hiperparámetros base: LR(max_iter=1000), SVM(probability=True), RF(random_state)
# 4. Validación: StratifiedKFold con 5 splits
pipelines = {
    "LogReg": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=1000, random_state=42))
    ]),
    "RandomForest": Pipeline([
        ("clf", RandomForestClassifier(random_state=42))
    ]),
    "SVM": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", SVC(probability=True, random_state=42))
    ])
}

param_grids = {
    "LogReg": {
        "clf__C": [0.1, 1, 10, 100],
        "clf__penalty": ["l2", 'none'],   # <- usa None (sin comillas)
        "clf__solver": ["lbfgs", "newton-cg"],  # compatibles con l2 y None
    },
    "RandomForest": {
        "clf__n_estimators": [100, 200, 300, 400],
        "clf__max_depth": [None, 10, 20, 30],
        "clf__min_samples_split": [2, 5, 10],
        "clf__min_samples_leaf": [1, 2, 4],
    },
    "SVM": {
        "clf__C": [0.1, 1, 10, 100],
        "clf__gamma": ["scale", "auto", 0.1, 0.01],
        "clf__kernel": ["rbf", "poly"],
    },
}

"""## 5. Búsqueda de Hiperparámetros y Selección del Mejor Modelo"""

# Configura la validación cruzada estratificada con 5 splits, mezclando los datos y usando una semilla para reproducibilidad.
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# Diccionarios para almacenar los mejores modelos encontrados y los resultados de la validación cruzada.
best_models = {}
cv_results = {}

# Define el número de iteraciones (combinaciones de hiperparámetros) a probar para cada modelo en RandomizedSearchCV.
# Se ajusta para Logistic Regression (16) y RandomForest/SVM (20), o 20 por defecto.
n_iter_map = {"LogReg": 16, "RandomForest": 20, "SVM": 20}

print("=== BÚSQUEDA DE HIPERPARÁMETROS ===")
# Itera sobre cada nombre de pipeline definido anteriormente ("LogReg", "RandomForest", "SVM").
for name in pipelines.keys():
    print(f"\n🔍 Optimizando {name}...")
    # Inicializa RandomizedSearchCV:
    # - `pipelines[name]`: El pipeline a optimizar (modelo + pasos de preprocesamiento si existen).
    # - `param_grids[name]`: El diccionario con los hiperparámetros y sus rangos a explorar para el modelo actual.
    # - `cv=cv`: Usa la estrategia de validación cruzada definida (StratifiedKFold).
    # - `scoring='accuracy'`: La métrica a optimizar (precisión).
    # - `n_iter=n_iter_map.get(name, 20)`: Número de combinaciones de hiperparámetros a probar aleatoriamente.
    # - `random_state=42`: Semilla para el generador de números aleatorios, asegura que la búsqueda sea reproducible.
    # - `n_jobs=-1`: Utiliza todos los núcleos disponibles del procesador para acelerar la búsqueda.
    # - `error_score="raise"`: Si ocurre un error al evaluar una combinación, lanza una excepción en lugar de asignar NaN (opcional).
    search = RandomizedSearchCV(
        pipelines[name],
        param_grids[name],
        cv=cv,
        scoring='accuracy',
        n_iter=n_iter_map.get(name, 20, error_score=np.nan),   # <- usa el mapa
        random_state=42,
        n_jobs=-1,
        error_score="raise"  # opcional: si quieres que explote en lugar de NaN
    )
    # Ejecuta la búsqueda de hiperparámetros en el conjunto de entrenamiento.
    search.fit(X_train, y_train)
    # Almacena el mejor modelo encontrado (el que obtuvo la mejor puntuación promedio en CV) en el diccionario `best_models`.
    best_models[name] = search.best_estimator_
    # Almacena los resultados relevantes de la búsqueda en el diccionario `cv_results`.
    cv_results[name] = {
        'best_score': float(search.best_score_), # La mejor puntuación promedio de CV.
        'best_params': search.best_params_, # Los hiperparámetros que lograron la mejor puntuación.
        'cv_mean': float(search.cv_results_['mean_test_score'].mean()), # Promedio de todas las puntuaciones de CV.
        'cv_std': float(search.cv_results_['mean_test_score'].std()) # Desviación estándar de las puntuaciones de CV.
    }
    # Imprime los resultados de la búsqueda para el modelo actual.
    print(f"✅ Mejor puntuación: {search.best_score_:.4f}")
    print(f"✅ Mejores parámetros: {search.best_params_}")


# Comparación final de modelos:
# Crea un DataFrame para resumir los mejores resultados de CV de cada modelo.
results_df = pd.DataFrame({
    'Modelo': list(cv_results.keys()), # Nombres de los modelos.
    'CV Score Mean': [cv_results[name]['best_score'] for name in cv_results.keys()], # La mejor puntuación de CV para cada modelo.
    'CV Score Std': [cv_results[name]['cv_std'] for name in cv_results.keys()], # La desviación estándar de las puntuaciones de CV.
    'Best Params': [str(cv_results[name]['best_params'])[:80] + "..." for name in cv_results.keys()] # Los mejores parámetros (truncados para mejor visualización).
# Ordena el DataFrame por la mejor puntuación de CV en orden descendente.
}).sort_values('CV Score Mean', ascending=False)
# Muestra el DataFrame resultante.
results_df

best_model_name = results_df.iloc[0]['Modelo']
best_pipeline = best_models[best_model_name]
best_cv_score = cv_results[best_model_name]['best_score']
best_params = cv_results[best_model_name]['best_params']

print(f"\n🏆 MEJOR MODELO SELECCIONADO: {best_model_name}")
print(f"📈 Mejor CV Score: {best_cv_score:.4f}")
print(f"⚙️ Parámetros óptimos: {best_params}")

"""## 6. Entrenamiento Final y Evaluación Exhaustiva"""

print("=== ENTRENAMIENTO FINAL Y EVALUACIÓN ===")
# Entrena el mejor pipeline (seleccionado en el paso anterior) con el conjunto de entrenamiento completo.
best_pipeline.fit(X_train, y_train)

# Realiza predicciones en el conjunto de prueba.
y_pred = best_pipeline.predict(X_test)
# Realiza predicciones de probabilidad (si el modelo lo soporta).
y_proba = best_pipeline.predict_proba(X_test)
# Calcula la precisión del modelo en el conjunto de prueba.
acc = accuracy_score(y_test, y_pred)
print(f"🎯 Accuracy (Test): {acc:.4f}")

print("\n📋 REPORTE DE CLASIFICACIÓN:")
# Imprime el reporte de clasificación completo (precisión, recall, f1-score) por clase.
print(classification_report(y_test, y_pred, target_names=wine.target_names, digits=4))

# Visualiza la matriz de confusión.
plt.figure(figsize=(8, 6))
# Calcula la matriz de confusión.
cm = confusion_matrix(y_test, y_pred)
# Crea un objeto para mostrar la matriz de confusión.
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=wine.target_names)
# Dibuja la matriz de confusión con un mapa de colores 'Blues' y valores enteros.
disp.plot(cmap='Blues', values_format='d')
plt.title(f'Matriz de Confusión - {best_model_name}', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

# Visualiza las curvas ROC por clase.
plt.figure(figsize=(10, 8))
# Codifica las etiquetas de prueba a formato binario para las curvas ROC.
# Itera sobre cada clase para trazar su curva ROC.
for i, cls in enumerate(wine.target_names):
    # Dibuja la curva ROC para la clase actual.
    RocCurveDisplay.from_predictions(
        (y_test == i).astype(int), # Convierte las etiquetas a binario (1 si es la clase actual, 0 en caso contrario).
        y_proba[:, i], # Probabilidades de la clase actual.
        name=f"Clase {cls}", # Nombre de la curva.
        plot_chance_level=(i==0) # Dibuja la línea de azar solo para la primera clase.
    )
plt.title(f'Curvas ROC por Clase - {best_model_name}', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

# Calcula el área bajo la curva ROC (AUC) macro promedio (estrategia One-vs-Rest).
auc_macro = roc_auc_score(y_test, y_proba, multi_class='ovr', average='macro')
print(f"📊 ROC-AUC Macro (OVR): {auc_macro:.4f}")

"""## 7. Análisis de Importancia de Características"""

print("=== ANÁLISIS DE IMPORTANCIA DE CARACTERÍSTICAS ===")

# Se verifica qué modelo fue seleccionado como el mejor para aplicar el método de importancia de características adecuado.
if best_model_name == "RandomForest":
    # Si el mejor modelo es RandomForest, se utiliza el atributo feature_importances_ del clasificador.
    feature_importances = best_pipeline.named_steps['clf'].feature_importances_
    # Se crea un DataFrame para mostrar las importancias junto con los nombres originales y traducidos de las características.
    importance_df = pd.DataFrame({
        'Característica': X.columns,
        'Importancia': feature_importances,
        'Nombre_ES': [column_titles_es.get(col, col) for col in X.columns]
    }).sort_values('Importancia', ascending=False) # Se ordena por importancia descendente.
    display(importance_df) # Se muestra el DataFrame.
    # Se crea un gráfico de barras para visualizar la importancia de las características.
    plt.figure(figsize=(12, 8))
    sns.barplot(data=importance_df, x='Importancia', y='Característica')
    plt.title('Importancia de Características - RandomForest', fontsize=14, pad=20)
    plt.xlabel('Importancia (Gini)')
    plt.tight_layout()
    plt.show()

elif best_model_name == "LogReg":
    # Si el mejor modelo es LogisticRegression, se utilizan los coeficientes del modelo.
    # Se toma el valor absoluto promedio de los coeficientes a través de las clases para obtener una medida de importancia.
    coefficients = best_pipeline.named_steps['clf'].coef_
    avg_importance = np.mean(np.abs(coefficients), axis=0)
    # Se crea un DataFrame similar al de RandomForest.
    importance_df = pd.DataFrame({
        'Característica': X.columns,
        'Importancia': avg_importance,
        'Nombre_ES': [column_titles_es.get(col, col) for col in X.columns]
    }).sort_values('Importancia', ascending=False) # Se ordena por importancia descendente.
    display(importance_df) # Se muestra el DataFrame.
    # Se crea un gráfico de barras para visualizar la importancia de las características.
    plt.figure(figsize=(12, 8))
    sns.barplot(data=importance_df, x='Importancia', y='Característica')
    plt.title('Importancia de Características - Regresión Logística', fontsize=14, pad=20)
    plt.xlabel('Importancia (|coef| promedio)')
    plt.tight_layout()
    plt.show()

else:
    # Si el mejor modelo es SVM (u otro que no tenga feature_importances_), se calcula la importancia por permutación.
    print("🔍 Calculando importancia por permutación...")
    # Se calcula la importancia por permutación en el conjunto de prueba.
    perm_importance = permutation_importance(
        best_pipeline, X_test, y_test,
        n_repeats=10, random_state=42, n_jobs=-1 # Se repite 10 veces y se usan todos los núcleos disponibles.
    )
    # Se crea un DataFrame con la importancia promedio calculada.
    importance_df = pd.DataFrame({
        'Característica': X.columns,
        'Importancia': perm_importance.importances_mean,
        'Nombre_ES': [column_titles_es.get(col, col) for col in X.columns]
    }).sort_values('Importancia', ascending=False) # Se ordena por importancia descendente.
    display(importance_df) # Se muestra el DataFrame.
    # Se crea un gráfico de barras para visualizar la importancia de las características.
    plt.figure(figsize=(12, 8))
    sns.barplot(data=importance_df, x='Importancia', y='Característica')
    plt.title('Importancia de Características - Permutación', fontsize=14, pad=20)
    plt.xlabel('Disminución promedio en accuracy') # Se especifica la métrica utilizada.
    plt.tight_layout()
    plt.show()

"""## 8. Persistencia del Modelo y Recursos"""

# ==== 8. Persistencia del Modelo y Recursos ====

# Crea el directorio 'model' si no existe para guardar los archivos.
os.makedirs("model", exist_ok=True)
# Define la ruta donde se guardará el mejor modelo, incluyendo el nombre del modelo y una versión.
model_path = f"model/best_model_{best_model_name}_v2.joblib"
# Guarda el objeto pipeline del mejor modelo en el archivo especificado usando joblib.
joblib.dump(best_pipeline, model_path)

# Crea un diccionario con metadatos relevantes sobre el modelo entrenado.
model_metadata = {
    'model_name': best_model_name, # Nombre del mejor modelo.
    'accuracy_test': float(acc), # Accuracy en el conjunto de prueba.
    'auc_macro': float(auc_macro), # ROC-AUC macro promedio.
    'best_parameters': best_params, # Parámetros óptimos encontrados.
    'feature_importances': importance_df.to_dict('records'), # Importancia de características como lista de diccionarios.
    'timestamp': pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S"), # Fecha y hora de guardado.
    'version': '2.0' # Versión del modelo/notebook.
}

# Guarda los metadatos en un archivo JSON.
with open('model/model_metadata.json', 'w') as f:
    json.dump(model_metadata, f, indent=2) # Usa indent=2 para un formato legible.

# Confirma que el modelo y los metadatos han sido guardados.
print("✅ MODELO Y METADATOS GUARDADOS")
print(f"📁 Modelo: {model_path}")
print(f"📁 Metadatos: model/model_metadata.json")

# ==== 9. Ejemplo de Inferencia con el Modelo Guardado ====

# Define una función para realizar predicciones con un nuevo sample.
# Toma el modelo entrenado y los datos del sample como entrada.
def predict_new_sample(model, sample_data):
    # Verifica si el modelo tiene el método predict_proba (para obtener probabilidades).
    if hasattr(model, 'predict_proba'):
        # Si lo tiene, predice las probabilidades y la clase.
        proba = model.predict_proba(sample_data)
        prediction = model.predict(sample_data)
        return prediction, proba # Retorna la predicción y las probabilidades.
    else:
        # Si no tiene predict_proba, solo predice la clase.
        prediction = model.predict(sample_data)
        return prediction, None # Retorna solo la predicción.

print("\n🎯 EJEMPLO DE INFERENCIA:")
# Selecciona una muestra del conjunto de prueba para el ejemplo.
sample_idx = 0 # Índice de la muestra (la primera).
sample = X_test.iloc[sample_idx:sample_idx+1] # Extrae la fila como DataFrame (necesario para el pipeline).
true_class = y_test.iloc[sample_idx] # Obtiene la clase real de la muestra.
# Realiza la predicción con el mejor pipeline.
pred_class = best_pipeline.predict(sample)[0] # Obtiene la clase predicha.
pred_proba = best_pipeline.predict_proba(sample)[0] # Obtiene las probabilidades predichas.

# Imprime los resultados del ejemplo.
print(f"Muestra: {sample_idx}")
print(f"Clase real: {true_class} ({wine.target_names[true_class]})") # Muestra la clase real con su nombre.
print(f"Clase predicha: {pred_class} ({wine.target_names[pred_class]})") # Muestra la clase predicha con su nombre.
print(f"Probabilidades: {np.round(pred_proba, 3)}") # Muestra las probabilidades redondeadas.

"""## 9. Plantilla de Revisión por Pares (Completar)

**Nombre del/la revisor/a:** ___________________  
**Fecha de revisión:** __________________

**Fortalezas observadas:**
- [ ] EDA avanzado con visualizaciones útiles.
- [ ] Uso de validación cruzada y búsqueda de hiperparámetros.
- [ ] Evaluación exhaustiva con ROC-AUC y matriz de confusión.
- [ ] Pipeline reproducible y documentación clara.

**Áreas de mejora:**
- [ ] Analizar impacto de normalización en cada modelo.
- [ ] Comparar otras métricas (macro F1, balanced accuracy).
- [ ] Probar otros clasificadores (XGBoost/LightGBM si se permite).
- [ ] Reducir dimensionalidad (PCA) y comparar.

**Ajustes realizados por mí (autor/a) después de la revisión:**
-  
-  
-  
"""